In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

dhcp-10-31-130-63.dyn.MIT.EDU
/Users/akarshkumar0101/synthetic-mdps/src
/Users/akarshkumar0101/miniconda3/bin/python


In [3]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [4]:
import jax
import jax.numpy as jnp
from jax.random import split

import flax
import flax.linen as nn
from flax.training.train_state import TrainState

import optax

In [36]:
rng = jax.random.PRNGKey(0)

In [57]:
dataset = dict(
    obs=repeat(jnp.arange(1000), "t -> n t", n=100),
    act=100+repeat(jnp.arange(1000), "t -> n t", n=100),
    rew=200+repeat(jnp.arange(1000), "t -> n t", n=100),
)

print(jax.tree_map(lambda x: x.shape, dataset))


{'act': (100, 1000), 'obs': (100, 1000), 'rew': (100, 1000)}


In [58]:
def preprocess_dataset(ds):
    ds_prv = {f"{k}_prv": v[:, :-2] for k, v in ds.items()}
    ds_now = {f"{k}": v[:, 1:-1] for k, v in ds.items()}
    ds_nxt = {f"{k}_nxt": v[:, 2:] for k, v in ds.items()}
    return {**ds_prv, **ds_now, **ds_nxt}

dataset = preprocess_dataset(dataset)

In [89]:
def sample_batch_segments_from_dataset(_rng, dataset, batch_size, n_segs, seg_len):
    _rng1, _rng2 = split(_rng)
    n_e, n_t, *_ = dataset['obs'].shape
    i_e = jax.random.randint(_rng1, (batch_size, 1, 1), minval=0, maxval=n_e)
    i_t = jax.random.randint(_rng2, (batch_size, n_segs, 1), minval=0, maxval=n_t - seg_len)
    i_t = i_t + jnp.arange(seg_len)
    batch = jax.tree_map(lambda x: x[i_e, i_t, ...], dataset)
    batch = jax.tree_map(lambda x: rearrange(x, 'b s t ... -> b (s t) ...'), batch)
    return batch

batch = sample_batch_segments_from_dataset(rng, dataset, 1, 1, 4)

In [90]:
print(jax.tree_map(lambda x: x.shape, batch))

{'act': (1, 4), 'act_nxt': (1, 4), 'act_prv': (1, 4), 'obs': (1, 4), 'obs_nxt': (1, 4), 'obs_prv': (1, 4), 'rew': (1, 4), 'rew_nxt': (1, 4), 'rew_prv': (1, 4)}


In [91]:
batch['obs']

Array([[819, 820, 821, 822]], dtype=int32)

In [93]:
batch['obs_nxt']

Array([[820, 821, 822, 823]], dtype=int32)